# Wellness Tech Analysis
> Author: Hannan Khan  
> Last updated: 02/20/2022

## Current Status

## Business Task

## Data Sources

## Data Cleaning
  
  
The database was created and loaded with the csv files in [this](https://github.com/hannankhan888/Data_Science_Portfolio/blob/main/Wellness_Tech_Company_Analysis_Case_Study/Database_Creator_Loader.ipynb) Jupyter Notebook.

### Loading Libraries

In [1]:
import sqlite3 as sql
from pprint import pprint
import os
import re

### Create Database Objects

In [2]:
data_dir = r"D:\Datasets\Fitabase_Data"
db_dir = data_dir + r"\\database.db"

# create a connection and cursor to the database:
db_con = sql.connect(db_dir)
cur = db_con.cursor()

# test the connection by deleting previously created tables, and listing the other tables:
# test if connection works:
query = ""
with open(r"SQL_queries\SQL_delete_ALL_created_tables.txt", mode="r") as file:
    query = file.readlines()
cur.executescript(''.join(query))

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:")
pprint(cur.fetchall())

Tables:
[('dailyActivity_merged',),
 ('dailyCalories_merged',),
 ('dailyIntensities_merged',),
 ('dailySteps_merged',),
 ('heartrate_seconds_merged',),
 ('hourlyCalories_merged',),
 ('hourlyIntensities_merged',),
 ('hourlySteps_merged',),
 ('minuteCaloriesNarrow_merged',),
 ('minuteCaloriesWide_merged',),
 ('minuteIntensitiesNarrow_merged',),
 ('minuteIntensitiesWide_merged',),
 ('minuteMETsNarrow_merged',),
 ('minuteSleep_merged',),
 ('minuteStepsNarrow_merged',),
 ('minuteStepsWide_merged',),
 ('sleepDay_merged',),
 ('weightLogInfo_merged',),
 ('minute4',)]


### Define Common Query Functions

In [3]:
def get_tables_and_cols(cur):
    """ Gets the tables and columns from a cursor object.
    RETURNS tables_cols: a list of tuples with table name and column name.
            tables_dict: a dict where every key is a table name and every
                        value is a list of column names."""
    get_cols = f"""
    SELECT
      m.name,
      p.name
    FROM 
      sqlite_master AS m
    JOIN 
      pragma_table_info(m.name) AS p
    WHERE m.type='table'
    """
    cur.execute(get_cols)
    tables_cols = cur.fetchall()

    # create tables dictionary:
    tables_set = set()
    tables_dict = {}
    for table,_ in tables_cols:
        tables_set.add(table)
    for table in tables_set:
        tables_dict[table] = []
        for tbl,col in tables_cols:
            if table==tbl:
                tables_dict[table].append(col)
    return tables_dict, tables_cols

def get_database_table_count(cur):
    """ Prints the number of tables from a cursor object."""
    
    cur.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table';")
    print("How many tables do we have:", cur.fetchall())

def get_all_pragma_tables(cur):
    """ Gets the pragma (schema) tables for all tables from the cursor object.
    Prints result as a table."""
    
    all_pragma_tables_query = """
    SELECT 
      m.name as table_name,
      p.*
    FROM 
      sqlite_master AS m
    JOIN 
      pragma_table_info(m.name) AS p
    ORDER BY 
      m.name, 
      p.cid
    """

    cur.execute(all_pragma_tables_query)
    print("|:::::::::::::::::::::::::::::::::::::ALL PRAGMA TABLES::::::::::::::::::::::::::::::::::::::::|")
    print("|{:30}|{:3}|{:25}|{:10}|{:7}|{:11}|{:2}|".format("table_name", "cid", "col_name", "type", "notnull", "dflt_values", "pk"))
    print("|",":"*92,"|")
    for blah in cur.fetchall():
        blah = [str(i) for i in blah]
        print("|{:30}|{:3}|{:25}|{:10}|{:7}|{:11}|{:2}|".format(blah[0], blah[1], blah[2], blah[3], blah[4], blah[5], blah[6]))

def get_table_col_example(cur):
    """ Gets the table name, column name, and an example data point from
    that column.
    Prints results in neat table."""
    
    _, tables_cols = get_tables_and_cols(cur)

    print("|{:30}|{:25}|{:25}|".format("TABLE", "COLUMN", "EXAMPLE"))
    print("|","="*80,"|")
    for table,col in tables_cols:
        get_example = f"""
        SELECT {col}
        FROM {table}
        LIMIT 1
        """
        cur.execute(get_example)
        print("|{:30}|{:25}|{:25}|".format(table, col, str(cur.fetchall()[0][0])))

def get_tables_with_num_rows(cur):
    """ Prints table names along with the number of rows in that table."""
    
    tables_dict, tables_cols = get_tables_and_cols(cur)
    print("Number of rows in each table:")
    for table in sorted(tables_dict.keys()):
        cur.execute(f"SELECT COUNT(*) FROM {table}")
        print("{:32}{:>15}".format(table, str(cur.fetchall())))

def get_table_cols_with_dtypes(cur, table_name):
    """Prints the table column names, along with their associated data types."""
    
    get_new_table_cols = f"""
    WITH minuteTables AS (
    SELECT m.name
    FROM 
      sqlite_master AS m
    JOIN 
      pragma_table_info(m.name) AS p
    WHERE
      m.name = '{table_name}'
    GROUP BY
      m.name
    )
    SELECT col AS columns, type AS dtype
    FROM (
        SELECT pti.name AS col, pti.type AS type
        FROM minuteTables AS t CROSS JOIN pragma_table_info(t.name) AS pti
        GROUP BY col
    );"""

    cur.execute(get_new_table_cols)
    print(f"`{table_name}` table cols:")
    pprint(cur.fetchall())

### Collect More Info On Our Tables/Schema
First we find the columns that are similar across ALL tables in the database.

In [4]:
# Lets check to see that we have 18 tables total:
get_database_table_count(cur)

How many tables do we have: [(19,)]


In [5]:
# what about common columns across all the tables?
similar_cols_all_tbls_query = """
WITH tables AS (SELECT name FROM sqlite_master WHERE type = 'table')
SELECT col AS columns
FROM (
    SELECT pti.name AS col
    FROM tables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(*) = (SELECT COUNT(*) FROM tables)
);
"""
cur.execute(similar_cols_all_tbls_query)
print("Common columns:", cur.fetchall())

Common columns: [('Id',)]


In [6]:
# double check to make sure ALL tables have the 'Id' column:
id_check_query = """
WITH tables AS (SELECT name FROM sqlite_master WHERE type = 'table')
SELECT col AS columns,
COUNT(*) AS num_tables
FROM (
    SELECT pti.name AS col
    FROM tables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    )
WHERE col = 'Id';
"""
cur.execute(id_check_query)
print("How many tables have Id columns:", cur.fetchall())

How many tables have Id columns: [('Id', 19)]


In [7]:
# Look at all cols and their data types:
get_all_pragma_tables(cur)

|:::::::::::::::::::::::::::::::::::::ALL PRAGMA TABLES::::::::::::::::::::::::::::::::::::::::|
|table_name                    |cid|col_name                 |type      |notnull|dflt_values|pk|
| :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: |
|dailyActivity_merged          |0  |Id                       |INT       |1      |None       |0 |
|dailyActivity_merged          |1  |ActivityDate             |DATE      |1      |None       |0 |
|dailyActivity_merged          |2  |TotalSteps               |INT       |1      |None       |0 |
|dailyActivity_merged          |3  |TotalDistance            |DEC(5,5)  |1      |None       |0 |
|dailyActivity_merged          |4  |TrackerDistance          |DEC(5,5)  |1      |None       |0 |
|dailyActivity_merged          |5  |LoggedActivitiesDistance |DEC(5,5)  |1      |None       |0 |
|dailyActivity_merged          |6  |VeryActiveDistance       |DEC(5,5)  |1      |None       |0 |
|dailyActivity_merged         

We can also see an example from each column:

In [8]:
# get tables and get column name and example data:
get_table_col_example(cur)

|TABLE                         |COLUMN                   |EXAMPLE                  |
| ================================================================================ |
|dailyActivity_merged          |Id                       |1503960366               |
|dailyActivity_merged          |ActivityDate             |4/12/2016                |
|dailyActivity_merged          |TotalSteps               |13162                    |
|dailyActivity_merged          |TotalDistance            |8.5                      |
|dailyActivity_merged          |TrackerDistance          |8.5                      |
|dailyActivity_merged          |LoggedActivitiesDistance |0                        |
|dailyActivity_merged          |VeryActiveDistance       |1.87999999523163         |
|dailyActivity_merged          |ModeratelyActiveDistance |0.550000011920929        |
|dailyActivity_merged          |LightActiveDistance      |6.05999994277954         |
|dailyActivity_merged          |SedentaryActiveDistance  |0      

In [9]:
# lets see if each table has a date or time column:
datetime_query = """
SELECT 
  COUNT(*) AS num_tables_w_datetime
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  p.type IN ('DATE', 'TIME', 'DATETIME')
ORDER BY 
  m.name, 
  p.cid;
"""

cur.execute(datetime_query)
print("How many tables have date or time columns:", cur.fetchall())

How many tables have date or time columns: [(18,)]


In [10]:
# what are the column names of the time columns?
datetime_query_two = """
SELECT m.name, p.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  p.type IN ('DATE', 'TIME', 'DATETIME')
ORDER BY 
  m.name, 
  p.cid;
"""

cur.execute(datetime_query_two)
print("Here are the tables, along with names of the date/time columns:")
pprint(cur.fetchall())

Here are the tables, along with names of the date/time columns:
[('dailyActivity_merged', 'ActivityDate'),
 ('dailyCalories_merged', 'ActivityDay'),
 ('dailyIntensities_merged', 'ActivityDay'),
 ('dailySteps_merged', 'ActivityDay'),
 ('heartrate_seconds_merged', 'Time'),
 ('hourlyCalories_merged', 'ActivityHour'),
 ('hourlyIntensities_merged', 'ActivityHour'),
 ('hourlySteps_merged', 'ActivityHour'),
 ('minuteCaloriesNarrow_merged', 'ActivityMinute'),
 ('minuteCaloriesWide_merged', 'ActivityHour'),
 ('minuteIntensitiesNarrow_merged', 'ActivityMinute'),
 ('minuteIntensitiesWide_merged', 'ActivityHour'),
 ('minuteMETsNarrow_merged', 'ActivityMinute'),
 ('minuteSleep_merged', 'date'),
 ('minuteStepsNarrow_merged', 'ActivityMinute'),
 ('minuteStepsWide_merged', 'ActivityHour'),
 ('sleepDay_merged', 'SleepDay'),
 ('weightLogInfo_merged', 'Date')]


`ActivityDate` from the table `dailyActivity_merged` is the first of many exceptions to how this data is formatted. Lets take a peek:

In [11]:
datetime_head_query = """
SELECT ActivityDate FROM dailyActivity_merged LIMIT 5;

"""

cur.execute(datetime_head_query)
pprint(cur.fetchall())

[('4/12/2016',), ('4/13/2016',), ('4/14/2016',), ('4/15/2016',), ('4/16/2016',)]


It seems that `ActivityDate` is just mm/dd/yyyy.  
Lets see what happens when we only want daily tables from our dataset.

In [12]:
get_daily_tables_query = """
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  INSTR(LOWER(m.name), 'day') OR
  INSTR(LOWER(m.name), 'daily')
GROUP BY
  m.name;
"""

cur.execute(get_daily_tables_query)
print("Here are the DAILY tables:")
pprint(cur.fetchall())

Here are the DAILY tables:
[('dailyActivity_merged',),
 ('dailyCalories_merged',),
 ('dailyIntensities_merged',),
 ('dailySteps_merged',),
 ('sleepDay_merged',)]


Let's see what columns are shared across the daily tables:

In [13]:
similar_daily_cols_query = """
WITH dailyTables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  INSTR(LOWER(m.name), 'day') OR
  INSTR(LOWER(m.name), 'daily')
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM dailyTables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(*) = (SELECT COUNT(*) FROM dailyTables)
);
"""

cur.execute(similar_daily_cols_query)
print("Columns similar across our DAILY tables include:", cur.fetchall())

Columns similar across our DAILY tables include: [('Id', 'INT')]


Let's gather the same info about the `hourly` and `minute` columns:

In [14]:
# hourly
get_hourly_tables_query = """
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  INSTR(LOWER(m.name), 'hourly')
GROUP BY
  m.name;
"""
cur.execute(get_hourly_tables_query)
print("Here are the HOURLY tables:")
pprint(cur.fetchall())

# common columns across hourly tables:
similar_hourly_cols_query = """
WITH hourlyTables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  INSTR(LOWER(m.name), 'hourly')
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM hourlyTables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(*) = (SELECT COUNT(*) FROM hourlyTables)
);
"""
cur.execute(similar_hourly_cols_query)
print("Columns similar across ALL HOURLY tables include:", cur.fetchall())


# minute
get_minute_tables_query = """
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  INSTR(LOWER(m.name), 'minute')
GROUP BY
  m.name;
"""
cur.execute(get_minute_tables_query)
print("\n\nHere are the MINUTE tables:")
pprint(cur.fetchall())

# common columns across hourly tables:
similar_minute_cols_query = """
WITH minuteTables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  INSTR(LOWER(m.name), 'minute')
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM minuteTables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(*) = (SELECT COUNT(*) FROM minuteTables)
);
"""
cur.execute(similar_minute_cols_query)
print("Columns similar across ALL MINUTE tables include:", cur.fetchall())

Here are the HOURLY tables:
[('hourlyCalories_merged',),
 ('hourlyIntensities_merged',),
 ('hourlySteps_merged',)]
Columns similar across ALL HOURLY tables include: [('ActivityHour', 'DATETIME'), ('Id', 'INT')]


Here are the MINUTE tables:
[('minute4',),
 ('minuteCaloriesNarrow_merged',),
 ('minuteCaloriesWide_merged',),
 ('minuteIntensitiesNarrow_merged',),
 ('minuteIntensitiesWide_merged',),
 ('minuteMETsNarrow_merged',),
 ('minuteSleep_merged',),
 ('minuteStepsNarrow_merged',),
 ('minuteStepsWide_merged',)]
Columns similar across ALL MINUTE tables include: [('Id', 'INT')]


### Creating A `Daily` Table
Now, welook at similar columns between dailyActivity_merged and dailyCalories_merged. Then, we use those columns to join those tables into a new table, `daily`, which we will be adding all of our daily data to.  
*NOTE:* Both tables have a date column, activityDate and activityDay, which we can also join on. These columns however do not show up on the similar columns query.

In [15]:
get_table_cols_with_dtypes(cur, "dailyActivity_merged")
get_table_cols_with_dtypes(cur, "dailyCalories_merged")

# look at similar columns between dailyActivity_merged and dailyCalories_merged:
similar_query = """
WITH twotables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  m.name IN ("dailyActivity_merged", "dailyCalories_merged")
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM twotables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(t.name) >=2
);
"""
cur.execute(similar_query)
print("Similar columns across dailyActivity_merged and dailyCalories_merged:", cur.fetchall())

# now we can join those two into one table:
create_daily_table_script = """
CREATE TABLE IF NOT EXISTS daily AS
    SELECT
        a.*
    FROM
        dailyActivity_merged AS a
    INNER JOIN
        dailyCalories_merged AS c
    ON
        a.Id = c.Id AND
        a.ActivityDate = c.ActivityDay AND
        a.Calories = c.Calories;
"""
cur.executescript(create_daily_table_script)
print("created table: `daily`.", cur.fetchall())

get_new_table_cols = """
WITH minuteTables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  m.name = 'daily'
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM minuteTables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
);"""

cur.execute(get_new_table_cols)
print("`daily` table cols:")
pprint(cur.fetchall())

`dailyActivity_merged` table cols:
[('ActivityDate', 'DATE'),
 ('Calories', 'INT'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'DEC(5,5)'),
 ('LightlyActiveMinutes', 'INT'),
 ('LoggedActivitiesDistance', 'DEC(5,5)'),
 ('ModeratelyActiveDistance', 'DEC(5,5)'),
 ('SedentaryActiveDistance', 'DEC(5,5)'),
 ('SedentaryMinutes', 'INT'),
 ('TotalDistance', 'DEC(5,5)'),
 ('TotalSteps', 'INT'),
 ('TrackerDistance', 'DEC(5,5)'),
 ('VeryActiveDistance', 'DEC(5,5)'),
 ('VeryActiveMinutes', 'INT')]
`dailyCalories_merged` table cols:
[('ActivityDay', 'DATE'), ('Calories', 'INT'), ('Id', 'INT')]
Similar columns across dailyActivity_merged and dailyCalories_merged: [('Calories', 'INT'), ('Id', 'INT')]
created table: `daily`. []
`daily` table cols:
[('ActivityDate', 'NUM'),
 ('Calories', 'INT'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'NUM'),
 ('LightlyActiveMinutes', 'INT'),
 ('LoggedActivitiesDistance', 'NUM'),
 ('ModeratelyActiveDistanc

Now we add dailyIntensities_merged to our daily table. We will follow the same process as before, finding which cols they share, and then joining on those columns.  
*NOTE:* They both share a date column as ActivityDate and ActivityDay.

In [16]:
get_table_cols_with_dtypes(cur, "dailyIntensities_merged")
get_table_cols_with_dtypes(cur, "daily")

# look at similar columns between dailyIntensities_merged  and daily:
similar_query = """
WITH twotables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  m.name IN ("dailyIntensities_merged", "daily")
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM twotables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(t.name) >=2
);
"""
cur.execute(similar_query)
print("Similar columns across dailyIntensities_merged  and daily:")
pprint(cur.fetchall())

# now we can join those two into one table:
create_daily_table_script = """
CREATE TABLE IF NOT EXISTS daily2 AS
    SELECT
        d.*
    FROM
        daily AS d
    JOIN
        dailyIntensities_merged AS i
    ON
        d.Id = i.Id AND
        d.ActivityDate = i.ActivityDay AND
        d.FairlyActiveMinutes = i.FairlyActiveMinutes AND
        d.LightActiveDistance = i.LightActiveDistance AND
        d.LightlyActiveMinutes = i.LightlyActiveMinutes AND
        d.ModeratelyActiveDistance = i.ModeratelyActiveDistance AND
        d.SedentaryActiveDistance = i.SedentaryActiveDistance AND
        d.SedentaryMinutes = i.SedentaryMinutes AND
        d.VeryActiveDistance = i.VeryActiveDistance AND
        d.VeryActiveMinutes = i.VeryActiveMinutes;
"""
cur.executescript(create_daily_table_script)
print("created table: `daily2`.", cur.fetchall())

get_table_cols_with_dtypes(cur, "daily2")

`dailyIntensities_merged` table cols:
[('ActivityDay', 'DATE'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'DEC(5,5)'),
 ('LightlyActiveMinutes', 'INT'),
 ('ModeratelyActiveDistance', 'DEC(5,5)'),
 ('SedentaryActiveDistance', 'DEC(5,5)'),
 ('SedentaryMinutes', 'INT'),
 ('VeryActiveDistance', 'DEC(5,5)'),
 ('VeryActiveMinutes', 'INT')]
`daily` table cols:
[('ActivityDate', 'NUM'),
 ('Calories', 'INT'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'NUM'),
 ('LightlyActiveMinutes', 'INT'),
 ('LoggedActivitiesDistance', 'NUM'),
 ('ModeratelyActiveDistance', 'NUM'),
 ('SedentaryActiveDistance', 'NUM'),
 ('SedentaryMinutes', 'INT'),
 ('TotalDistance', 'NUM'),
 ('TotalSteps', 'INT'),
 ('TrackerDistance', 'NUM'),
 ('VeryActiveDistance', 'NUM'),
 ('VeryActiveMinutes', 'INT')]
Similar columns across dailyIntensities_merged  and daily:
[('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'NUM'),
 ('LightlyActiveMinut

Let's check the number of rows for each table:

In [17]:
get_tables_with_num_rows(cur)

Number of rows in each table:
daily                                  [(940,)]
daily2                                 [(940,)]
dailyActivity_merged                   [(940,)]
dailyCalories_merged                   [(940,)]
dailyIntensities_merged                [(940,)]
dailySteps_merged                      [(940,)]
heartrate_seconds_merged           [(2483658,)]
hourlyCalories_merged                [(22099,)]
hourlyIntensities_merged             [(22099,)]
hourlySteps_merged                   [(22099,)]
minute4                             [(125903,)]
minuteCaloriesNarrow_merged        [(1325580,)]
minuteCaloriesWide_merged            [(21645,)]
minuteIntensitiesNarrow_merged     [(1325580,)]
minuteIntensitiesWide_merged         [(21645,)]
minuteMETsNarrow_merged            [(1325580,)]
minuteSleep_merged                  [(188521,)]
minuteStepsNarrow_merged           [(1325580,)]
minuteStepsWide_merged               [(21645,)]
sleepDay_merged                        [(413,)]
weightLogI

Now we add dailySteps_merged to our daily2 table. We will follow the same process as before, finding which cols they share, and then joining on those columns.  
*NOTE:* They both share a date column as ActivityDate and ActivityDay, and a steps column as TotalSteps and StepTotal respectively.

In [18]:
get_table_cols_with_dtypes(cur, "dailySteps_merged")
get_table_cols_with_dtypes(cur, "daily2")

# look at similar columns between dailySteps_merged  and daily2:
similar_query = """
WITH twotables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  m.name IN ("dailySteps_merged", "daily2")
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM twotables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(t.name) >=2
);
"""
cur.execute(similar_query)
print("Similar columns across dailySteps_merged and daily2:")
pprint(cur.fetchall())

# now we can join those two into one table:
create_daily_table_script = """
CREATE TABLE IF NOT EXISTS daily3 AS
    SELECT
        d.*
    FROM
        daily2 AS d
    JOIN
        dailySteps_merged AS s
    ON
        d.Id = s.Id AND
        d.ActivityDate = s.ActivityDay;
"""
cur.executescript(create_daily_table_script)
print("created table: `daily3`.", cur.fetchall())

get_table_cols_with_dtypes(cur, "daily3")

`dailySteps_merged` table cols:
[('ActivityDay', 'DATE'), ('Id', 'INT'), ('StepTotal', 'INT')]
`daily2` table cols:
[('ActivityDate', 'NUM'),
 ('Calories', 'INT'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'NUM'),
 ('LightlyActiveMinutes', 'INT'),
 ('LoggedActivitiesDistance', 'NUM'),
 ('ModeratelyActiveDistance', 'NUM'),
 ('SedentaryActiveDistance', 'NUM'),
 ('SedentaryMinutes', 'INT'),
 ('TotalDistance', 'NUM'),
 ('TotalSteps', 'INT'),
 ('TrackerDistance', 'NUM'),
 ('VeryActiveDistance', 'NUM'),
 ('VeryActiveMinutes', 'INT')]
Similar columns across dailySteps_merged and daily2:
[('Id', 'INT')]
created table: `daily3`. []
`daily3` table cols:
[('ActivityDate', 'NUM'),
 ('Calories', 'INT'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'NUM'),
 ('LightlyActiveMinutes', 'INT'),
 ('LoggedActivitiesDistance', 'NUM'),
 ('ModeratelyActiveDistance', 'NUM'),
 ('SedentaryActiveDistance', 'NUM'),
 ('SedentaryMinutes', 'INT'),
 ('TotalD

In [19]:
get_tables_with_num_rows(cur)

Number of rows in each table:
daily                                  [(940,)]
daily2                                 [(940,)]
daily3                                 [(940,)]
dailyActivity_merged                   [(940,)]
dailyCalories_merged                   [(940,)]
dailyIntensities_merged                [(940,)]
dailySteps_merged                      [(940,)]
heartrate_seconds_merged           [(2483658,)]
hourlyCalories_merged                [(22099,)]
hourlyIntensities_merged             [(22099,)]
hourlySteps_merged                   [(22099,)]
minute4                             [(125903,)]
minuteCaloriesNarrow_merged        [(1325580,)]
minuteCaloriesWide_merged            [(21645,)]
minuteIntensitiesNarrow_merged     [(1325580,)]
minuteIntensitiesWide_merged         [(21645,)]
minuteMETsNarrow_merged            [(1325580,)]
minuteSleep_merged                  [(188521,)]
minuteStepsNarrow_merged           [(1325580,)]
minuteStepsWide_merged               [(21645,)]
sleepDay_m

Lastly, we will add the `sleepDay_merged` table to our `daily3` to create `daily4`.

In [20]:
get_table_cols_with_dtypes(cur, "sleepDay_merged")
get_table_cols_with_dtypes(cur, "daily3")

# look at similar columns between dailySteps_merged  and daily2:
similar_query = """
WITH twotables AS (
SELECT m.name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
WHERE
  m.name IN ("sleepDay_merged", "daily3")
GROUP BY
  m.name
)
SELECT col AS columns, type AS dtype
FROM (
    SELECT pti.name AS col, pti.type AS type
    FROM twotables AS t CROSS JOIN pragma_table_info(t.name) AS pti
    GROUP BY col
    HAVING COUNT(t.name) >=2
);
"""
cur.execute(similar_query)
print("Similar columns across sleepDay_merged and daily3:")
pprint(cur.fetchall())

# now we can join those two into one table:
create_daily_table_script = """
CREATE TABLE IF NOT EXISTS daily4 AS
    SELECT
        d.*,
        s.SleepDay,
        s.TotalMinutesAsleep,
        s.TotalSleepRecords,
        s.TotalTimeInBed
    FROM
        daily3 AS d
    JOIN
        sleepDay_merged AS s
    ON
        d.Id = s.Id
"""
cur.executescript(create_daily_table_script)
print("created table: `daily4`.", cur.fetchall())

get_table_cols_with_dtypes(cur, "daily4")

`sleepDay_merged` table cols:
[('Id', 'INT'),
 ('SleepDay', 'DATETIME'),
 ('TotalMinutesAsleep', 'INT'),
 ('TotalSleepRecords', 'INT'),
 ('TotalTimeInBed', 'INT')]
`daily3` table cols:
[('ActivityDate', 'NUM'),
 ('Calories', 'INT'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'NUM'),
 ('LightlyActiveMinutes', 'INT'),
 ('LoggedActivitiesDistance', 'NUM'),
 ('ModeratelyActiveDistance', 'NUM'),
 ('SedentaryActiveDistance', 'NUM'),
 ('SedentaryMinutes', 'INT'),
 ('TotalDistance', 'NUM'),
 ('TotalSteps', 'INT'),
 ('TrackerDistance', 'NUM'),
 ('VeryActiveDistance', 'NUM'),
 ('VeryActiveMinutes', 'INT')]
Similar columns across sleepDay_merged and daily3:
[('Id', 'INT')]
created table: `daily4`. []
`daily4` table cols:
[('ActivityDate', 'NUM'),
 ('Calories', 'INT'),
 ('FairlyActiveMinutes', 'INT'),
 ('Id', 'INT'),
 ('LightActiveDistance', 'NUM'),
 ('LightlyActiveMinutes', 'INT'),
 ('LoggedActivitiesDistance', 'NUM'),
 ('ModeratelyActiveDistance', 'NUM'),
 ('Sedent

Lets clean up our temp tables and rename `daily4` to `daily`.

In [21]:
q = """
DROP TABLE IF EXISTS daily;
DROP TABLE IF EXISTS daily2;
DROP TABLE IF EXISTS daily3;
ALTER TABLE daily4 RENAME TO daily;
"""
cur.executescript(q)
print(cur.fetchall())

[]


In [22]:
get_tables_with_num_rows(cur)

Number of rows in each table:
daily                                [(12441,)]
dailyActivity_merged                   [(940,)]
dailyCalories_merged                   [(940,)]
dailyIntensities_merged                [(940,)]
dailySteps_merged                      [(940,)]
heartrate_seconds_merged           [(2483658,)]
hourlyCalories_merged                [(22099,)]
hourlyIntensities_merged             [(22099,)]
hourlySteps_merged                   [(22099,)]
minute4                             [(125903,)]
minuteCaloriesNarrow_merged        [(1325580,)]
minuteCaloriesWide_merged            [(21645,)]
minuteIntensitiesNarrow_merged     [(1325580,)]
minuteIntensitiesWide_merged         [(21645,)]
minuteMETsNarrow_merged            [(1325580,)]
minuteSleep_merged                  [(188521,)]
minuteStepsNarrow_merged           [(1325580,)]
minuteStepsWide_merged               [(21645,)]
sleepDay_merged                        [(413,)]
weightLogInfo_merged                    [(67,)]


### Creating The `hourly`, `minuteNarrow`, `minuteWide` Tables
Since this process has taken so long, we will write a separate script for combining the hourly and minutes tables into the tables `hourly` and `minute`.  
The hourly table creation script is [SQL_create_hourly_table.txt](https://github.com/hannankhan888/Data_Science_Portfolio/blob/main/Wellness_Tech_Company_Analysis_Case_Study/SQL_queries/SQL_create_hourly_table.txt).  
The minute tables present a unique problem. The data is presented in both narrow and wide format. The `narrow` format contains data by the minute. The `wide` format contains data by the hour, with 60 columns, each representing the calories burned within that minute of that hour.  
We will use data from the `narrow` tables to create our `minuteNarrow` table.
The minute table creation script is [SQL_create_minuteNarrow_table.txt](https://github.com/hannankhan888/Data_Science_Portfolio/blob/main/Wellness_Tech_Company_Analysis_Case_Study/SQL_queries/SQL_create_minuteNarrow_table.txt).  
We will also go ahead and combine the `wide` tables in case we need to use them later, into our `minuteWide` table. The minuteWide table creation script is [SQL_create_minuteWide_table.txt](https://github.com/hannankhan888/Data_Science_Portfolio/blob/main/Wellness_Tech_Company_Analysis_Case_Study/SQL_queries/SQL_create_minuteWide_table.txt).

In [23]:
# executing create hourly script:
query = ""
with open(r"SQL_queries\SQL_create_hourly_table.txt", mode="r") as file:
    query = file.readlines()
cur.executescript(''.join(query))
print(cur.fetchall())

# executing create minuteNarrow script:
query = ""
with open(r"SQL_queries\SQL_create_minuteNarrow_table.txt", mode="r") as file:
    query = file.readlines()
cur.executescript(''.join(query))
print(cur.fetchall())

# executing create minuteWide script:
query = ""
with open(r"SQL_queries\SQL_create_minuteWide_table.txt", mode="r") as file:
    query = file.readlines()
cur.executescript(''.join(query))
print(cur.fetchall())

# executing delete extra tables script:
query = ""
with open(r"SQL_queries\SQL_keep_main_tables_only.txt", mode="r") as file:
    query = file.readlines()
cur.executescript(''.join(query))
print(cur.fetchall())

[]
[]
[]
[]


In [24]:
get_tables_with_num_rows(cur)

Number of rows in each table:
daily                                [(12441,)]
hourly                               [(22099,)]
minuteNarrow                        [(125903,)]
minuteWide                           [(21645,)]


### Creating A Dump Table For Our Database For Easy Recreation
Saving a SQL dump file will help us to quickly recreate our database in case it corrupts during our analysis. We can also treat this file as a backup.

In [27]:
dump_path = data_dir + r"\\main_tables_dump.sql"

with open(dump_path, 'w') as f:
    for line in db_con.iterdump():
        f.write(f"{line}\n")

## Data Preparation-Processing

## Analysis

### Analysis Summary

## Actions

## Appendix

### Definitions

### Files Used